In [2]:
import pandas as pd

In [84]:
from gensim.models import Word2Vec
#from gensim.utils import simple_preprocess
#from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import jieba
import re

In [79]:
train=[]
with open("train.tsv", "r", encoding="utf-8") as file:
    for line in file:
        parts = line.split("\t", 1) 
        first_part = parts[0]  
        rest_part = parts[1] if len(parts) > 1 else ""  
        rest_part = rest_part.replace('\n', '')
        train.append([first_part, rest_part])

In [80]:
train=pd.DataFrame(train[1:],columns=['label','train_text'],)

In [85]:
train.head()

,label,train_text
0,3,私募大佬抛弃了谁？
1,2,前上海上港球员孙祥的爱妻，晒出自己近日的自拍照
2,5,首台奇瑞瑞虎8到店，围观人群挤不动，网友：不到10万要卖疯
3,0,王羲之集字《圣教序》，很多字东倒西歪，如何理解其中的美感？
4,2,为什么明明詹姆斯最后体力不支，但是投那个绝杀球的时候，还能跳那么高？


In [93]:
labels = train["label"].tolist()  

In [95]:
corpus_tokenize=[]
for text in train["train_text"]:  
    text_processed = re.sub(r'[^\w\s]', '', text)
    seg_list = jieba.cut(text_processed,cut_all=False)
    corpus_tokenize.append(list(seg_list))

In [ ]:
#corpus_tokenized = [simple_preprocess(text) for text in corpus]

In [97]:
word2vec_model = Word2Vec(sentences=corpus_tokenize, vector_size=100, window=5, min_count=1, sg=1)

In [105]:
word2vec_model.wv.most_similar('微软')

[('谷歌', 0.8645645380020142),
 ('Google', 0.8510115742683411),
 ('开发者', 0.8479593992233276),
 ('Build', 0.8460357785224915),
 ('Windows', 0.8317628502845764),
 ('Alexa', 0.8185619115829468),
 ('AI', 0.8174707293510437),
 ('Cortana', 0.8117767572402954),
 ('大疆', 0.8085562586784363),
 ('小娜', 0.8014857769012451)]

In [119]:
word2vec_model.wv.most_similar(positive=['华为'], negative=['小米'],topn=5)

[('标准', 0.3255839943885803),
 ('短码', 0.32083553075790405),
 ('输', 0.3136298954486847),
 ('码', 0.31221768260002136),
 ('丢失', 0.3069092631340027)]

In [117]:
# 文本表示
def text_to_vector(text):
    vectors = []
    for word in text:
        if word in word2vec_model.wv:
            vectors.append(word2vec_model.wv[word])
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

In [118]:
X = np.array([text_to_vector(text) for text in corpus_tokenize])
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

classifier = SVC()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8073732116804215
